In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import csv

In [41]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD,Adam
from keras.utils import print_summary, to_categorical
import sys
import os

In [33]:
num_classes = 5
epochs = 10
learning_rate = 0.01
batch_size = 32
data_augmentation = True
DIM = 300
train_percentage = 0.7

### Read labels from csv file

In [18]:
with open('trainLabels.csv', 'r') as f:
    table = np.array(list(csv.reader(f, delimiter=',')))

### load images from folder

In [20]:
 def get_images(file_path,image_list,labels):
    file_names = []
    print(file_path)
    for root, dirs, files in os.walk(file_path):  
        for filename in files:
            file_names.append(filename)
    img_names = file_names
    #print(file_names[0][:-5])
    for name in file_names:
        try:
            index = np.where(table[:,0]==name[:-5])
            #print(index)
            labels.append(int(table[index][0,1]))
        except ValueError:
            img_names.remove(name)
            print("{} not available".format(name))
    for filename in img_names: #assuming jpeg
        im=Image.open(file_path+"//"+filename).resize((DIM,DIM), Image.ANTIALIAS)
        image_list.append(np.array(im,dtype=np.float32))
    return image_list, labels

### enter list of file names

In [21]:
file_paths = []
file_paths.append(os.getcwd()+"\\train_1")

In [22]:
image_list = []
labels = []
for path in file_paths:
    image_list,labels = get_images(path,image_list,labels)

C:\Users\k_tej\Desktop\retina\train_1


In [23]:
len(image_list),len(labels)

(71, 71)

### converting list of images into numpy array 

(71, 300, 300, 3)

### Converting them into numpy array 

In [29]:
image_list = np.array(image_list)
# one hot encoded numpy array
id_mtx = np.identity(num_classes,dtype=np.float32)
labels = id_mtx[labels]

(71, 300, 300, 3)


In [30]:
labels.shape,image_list.shape

((71, 5), (71, 300, 300, 3))

### dividing the data into train and test images

In [37]:
#split number
train_limit = int(labels.shape[0]*train_percentage)
test_limit = labels.shape[0]-train_limit
print(train_limit,test_limit)

49 22


In [38]:
(x_train, y_train) = (image_list[:train_limit,:,:,:],labels[:train_limit,:])
(x_test, y_test) =  (image_list[-test_limit:,:,:,:],labels[-test_limit:,:])

In [39]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

### Model code

In [42]:

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_trained_model.h5'

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/10


KeyboardInterrupt: 